#Домашнее задание 2. Бейзлайны и детерминированные алгоритмы item-item

In [ ]:
!pip install implicit==0.4.4

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [ ]:
#Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items_weights = items_weights.groupby('item_id')['sales_value'].sum().reset_index()
    recs = np.random.choice(np.log(items_weights.item_id), size=n, replace=False)
    
    return recs.tolist()

In [7]:
#%%time

# your_code
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(data_train, n=5))
result.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13.708725695072031, 16.354759075514277, 16.15..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[16.349550725667473, 12.250395704166838, 13.86..."


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров


In [9]:
# your_code
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [10]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()


In [11]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [12]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))



CPU times: user 3.94 s, sys: 52.2 ms, total: 3.99 s
Wall time: 3.98 s


In [13]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13.708725695072031, 16.354759075514277, 16.15...","[1347001, 1539347, 2470128, 16805950, 15831255]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[16.349550725667473, 12.250395704166838, 13.86...","[891148, 15682370, 6391421, 1047486, 1135848]"


In [14]:
%%time

items = data_train.item_id.unique()

result['random_5000_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))

CPU times: user 1.06 s, sys: 27.8 ms, total: 1.09 s
Wall time: 1.05 s


In [15]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,random_5000_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13.708725695072031, 16.354759075514277, 16.15...","[1347001, 1539347, 2470128, 16805950, 15831255]","[1013615, 1013167, 871680, 889692, 937626]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[16.349550725667473, 12.250395704166838, 13.86...","[891148, 15682370, 6391421, 1047486, 1135848]","[882138, 1137507, 1066097, 1044259, 10456568]"


- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [16]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [17]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)


In [18]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [19]:
%%time

model_1 = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model_2 = ItemItemRecommender(K=10, num_threads=4) # Увеличиваем K до 10 

model_3 = ItemItemRecommender(K=1, num_threads=4) # Уменьшаем K до 1


model_1.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

model_2.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

model_3.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 7.19 s, sys: 86.5 ms, total: 7.28 s
Wall time: 6.77 s


In [20]:
%%time

result['itemitem_1'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model_1.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

result['itemitem_2'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model_2.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

result['itemitem_3'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model_3.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

CPU times: user 246 ms, sys: 1.9 ms, total: 247 ms
Wall time: 263 ms


In [21]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,random_5000_recommendation,itemitem_1,itemitem_2,itemitem_3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13.708725695072031, 16.354759075514277, 16.15...","[1347001, 1539347, 2470128, 16805950, 15831255]","[1013615, 1013167, 871680, 889692, 937626]","[6666, 1082185, 981760, 1127831, 995242]","[6666, 1082185, 981760, 995242, 840361]","[6666, 1082185, 995242, 1029743, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[16.349550725667473, 12.250395704166838, 13.86...","[891148, 15682370, 6391421, 1047486, 1135848]","[882138, 1137507, 1066097, 1044259, 10456568]","[6666, 1082185, 981760, 1098066, 995242]","[6666, 1082185, 981760, 1098066, 826249]","[6666, 1082185, 1098066, 6534178, 826249]"


Метрики:

In [22]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [23]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [24]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0:weighted_random_recommendation
0.0007:random_recommendation
0.0052:random_5000_recommendation
0.1368:itemitem_1
0.1509:itemitem_2
0.1923:itemitem_3
